In [29]:
import numpy as np 
import pandas as pd 
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)



In [30]:
#ADDING THE DATASET

path =("D:\project\Dataset.csv")
data=pd.read_csv(path)


In [31]:
#
#data['source'].value_counts()

In [32]:
lines=data[data['source']=='indic2012']
lines.head(20)

,source,english_sentence,hindi_sentence
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
6,indic2012,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
8,indic2012,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
11,indic2012,Category: Religious Text,श्रेणी:धर्मग्रन्थ
12,indic2012,This period summarily is pepped up with devotion.,यह काल समग्रतः भक्ति भावना से ओतप्रोत काल है।
16,indic2012,"And now at present the naturecure, Ayurvedic and modern treatments are taking place through the government in Nepal.","हाल में नेपाल के हस्पताल सामन्यतया आयुर्वेद, प्राकृतिक चिकित्सा तथा आधुनिक चिकीत्सा करके सरकारी सेवा विद्यमान हे ।"
17,indic2012,Parliament time frame is 5 years and this will be dissolved before that.,लोकसभा की कार्यावधि 5 वर्ष है पर्ंतु इसे समय से पूर्व भंग किया जा सकता है
19,indic2012,"Extreme weather due to increased mortality; displacements and economic loss will be compounded through growing population. Although, temperate climate has some benefits out of it such as decreased mortality due to cold weather.",बढ़ती हुई मौतों displacements और आर्थिक नुकसान जो की अतिवादी मौसम (extreme weather)के कारण संभावित हैं बढती हुई जनसँख्या (growing population)के कारण और भी बदतर हो सकते हैं . हालांकि शीतोष्ण क्षेत्र में इसके कुछ फैदे भी हो सकते हैं जैसे की ठंड की वजह से कम मौतें होना .
22,indic2012,"Islam is the world's second-largest religion, after Christianity.",इस्लाम धर्म (الإسلام) ईसाई धर्म के बाद अनुयाइयों के आधार पर दुनिया का दूसरा सब से बड़ा धर्म है।


In [33]:
#Checking for null values
pd.isnull(lines).sum() 

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [34]:
#Dropping null values if any
lines=lines[~pd.isnull(lines['english_sentence'])]

In [35]:
#Dropping duplicates values if any
lines.drop_duplicates(inplace=True)

In [36]:
#Extracting Small Part of Dataset to cross check if any null values are present

lines=lines.sample(n=25000,random_state=42)
lines
#lines.shape


,source,english_sentence,hindi_sentence
43137,indic2012,Occurring in the environment recently increased carbon di Oxide (CO2) monthly CO2 measurements Dershat that if you saw all year get to see the seasonal changes are small; every year Paryaverton Northern Hemisphere (Northern Hemisphere) seasons in the Aswh At the end of the Northern Hemisphere crops have become more Bjne is the time get at the change of environment because the plants remove some CO2 charge,वातावरण में कार्बन डाइऑक्साइड में हाल ही में होने वाली बढोतरी ( CO2 ) .मासिक CO2 मापन यह दर्शाते हैं कि अगर सारे वर्ष को देखा जाए तो छोटे-छोटे मौसमी परिवर्तन देखने को मिलते हैं ; हर साल यह परिवर्तन उत्तरी गोलार्ध (Northern Hemisphere) में वसव्त मौसम के आख़िर में अधिक हो जाते हैं और जब उत्तरी गोलार्ध में फसलें बीजने का समय होता है तो यह परिवर्तन कम हो जाते हैं क्यूंकि पौधे वातावरण में से कुछ CO2 हटा लेते हैंI
107772,indic2012,Her father Aghoranath Chattopadhyay was a well-known scholar and mother was a poet who wrote in Bengali.,इनके पिता अघोरनाथ चट्टोपाध्याय एक नामी विद्वान तथा माँ कवयित्री थीं और बांग्ला में लिखती थीं ।
120507,indic2012,Nishadraj Guh served those three very well in Ringverpur.,ऋंगवेरपुर में निषादराज गुह ने तीनों की बहुत सेवा की।
28755,indic2012,"2. The ordinance that was rejected by Loksabha, can be passed again before six weeks period.",2. लोकसभा एक अध्यादेश को अस्वीकृत करने वाला प्रस्ताव 6 सप्ताह की अवधि समाप्त होने से पूर्व पास कर सकती है
86303,indic2012,Number of 18,अठारह की संख्या
...,...,...,...
117657,indic2012,"There are 6 political parties in India, But the Indian national congress is the most powerful in India",बहुदलिय राजनीतिक व्यवस्था वाले इस देश में ६ राष्ट्रीय स्तर के राजनीतिक दल हैं किंतु यहां की राजनीतिक व्यवस्था पर सर्वाधीक प्रभावी भारतीय राष्ट्रीय कांग्रेस रही है।
89633,indic2012,They had the experience of ruling.,उन्हें शासन करने का अनुभव है ।
127215,indic2012,Ram Kabir comprehensive nature of the concept and wanted to give a different,कबीर राम की अवधारणा को एक भिन्न और व्यापक स्वरूप देना चाहते थे।
67194,indic2012,"this country has many famous places like Agra, Aligad,Ayodhya,Berle,Me rut,Varanasi, (Banaras)Gorakhpur,Ghaziabad,Muradabad,Saharanpur, Faisalabad,Kanpur,",प्रदेश के अन्य महत्त्वपूर्ण शहर हैं आगरा अलीगढ अयोध्या बरेली मेरठ वाराणसी( बनारस) गोरखपुर गाज़ियाबाद मुरादाबाद सहारनपुर फ़ैज़ाबाद कानपुर।


In [37]:
#USING LAMBDA FUNCTIIONS

In [38]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [39]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [40]:
# Set of all special characters
exclude = set(string.punctuation) 
#print(exclude)

# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))


In [41]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
print(remove_digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

{48: None, 49: None, 50: None, 51: None, 52: None, 53: None, 54: None, 55: None, 56: None, 57: None}


In [42]:
# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())


lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [43]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [44]:
lines.head()

,source,english_sentence,hindi_sentence
43137,indic2012,occurring in the environment recently increased carbon di oxide co monthly co measurements dershat that if you saw all year get to see the seasonal changes are small every year paryaverton northern hemisphere northern hemisphere seasons in the aswh at the end of the northern hemisphere crops have become more bjne is the time get at the change of environment because the plants remove some co charge,START_ वातावरण में कार्बन डाइऑक्साइड में हाल ही में होने वाली बढोतरी co मासिक co मापन यह दर्शाते हैं कि अगर सारे वर्ष को देखा जाए तो छोटेछोटे मौसमी परिवर्तन देखने को मिलते हैं हर साल यह परिवर्तन उत्तरी गोलार्ध northern hemisphere में वसव्त मौसम के आख़िर में अधिक हो जाते हैं और जब उत्तरी गोलार्ध में फसलें बीजने का समय होता है तो यह परिवर्तन कम हो जाते हैं क्यूंकि पौधे वातावरण में से कुछ co हटा लेते हैंi _END
107772,indic2012,her father aghoranath chattopadhyay was a wellknown scholar and mother was a poet who wrote in bengali,START_ इनके पिता अघोरनाथ चट्टोपाध्याय एक नामी विद्वान तथा माँ कवयित्री थीं और बांग्ला में लिखती थीं । _END
120507,indic2012,nishadraj guh served those three very well in ringverpur,START_ ऋंगवेरपुर में निषादराज गुह ने तीनों की बहुत सेवा की। _END
28755,indic2012,the ordinance that was rejected by loksabha can be passed again before six weeks period,START_ लोकसभा एक अध्यादेश को अस्वीकृत करने वाला प्रस्ताव सप्ताह की अवधि समाप्त होने से पूर्व पास कर सकती है _END
86303,indic2012,number of,START_ अठारह की संख्या _END


In [45]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [57]:
len(all_eng_words)

30447

In [58]:
len(all_hindi_words)


20506

In [48]:
#Calculating length of each sentence
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [49]:
lines.head()


,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
43137,indic2012,occurring in the environment recently increased carbon di oxide co monthly co measurements dershat that if you saw all year get to see the seasonal changes are small every year paryaverton northern hemisphere northern hemisphere seasons in the aswh at the end of the northern hemisphere crops have become more bjne is the time get at the change of environment because the plants remove some co charge,START_ वातावरण में कार्बन डाइऑक्साइड में हाल ही में होने वाली बढोतरी co मासिक co मापन यह दर्शाते हैं कि अगर सारे वर्ष को देखा जाए तो छोटेछोटे मौसमी परिवर्तन देखने को मिलते हैं हर साल यह परिवर्तन उत्तरी गोलार्ध northern hemisphere में वसव्त मौसम के आख़िर में अधिक हो जाते हैं और जब उत्तरी गोलार्ध में फसलें बीजने का समय होता है तो यह परिवर्तन कम हो जाते हैं क्यूंकि पौधे वातावरण में से कुछ co हटा लेते हैंi _END,67,81
107772,indic2012,her father aghoranath chattopadhyay was a wellknown scholar and mother was a poet who wrote in bengali,START_ इनके पिता अघोरनाथ चट्टोपाध्याय एक नामी विद्वान तथा माँ कवयित्री थीं और बांग्ला में लिखती थीं । _END,17,19
120507,indic2012,nishadraj guh served those three very well in ringverpur,START_ ऋंगवेरपुर में निषादराज गुह ने तीनों की बहुत सेवा की। _END,9,12
28755,indic2012,the ordinance that was rejected by loksabha can be passed again before six weeks period,START_ लोकसभा एक अध्यादेश को अस्वीकृत करने वाला प्रस्ताव सप्ताह की अवधि समाप्त होने से पूर्व पास कर सकती है _END,15,21
86303,indic2012,number of,START_ अठारह की संख्या _END,2,5


In [50]:
#Lines having length greater than 30.
lines[lines['length_eng_sentence']>30].shape


(2030, 5)

In [59]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [52]:
lines.shape

(16970, 5)

In [53]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [62]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence']) #Target sentence having length 20
print(max_length_src)
print(max_length_tar)

20
20


In [64]:
input_words = sorted(list(all_eng_words))
#print(input_words)
target_words = sorted(list(all_hindi_words))
#print(target_words)
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens #Prints the total number of words


(30447, 20506)

In [73]:
#For zero padding
num_decoder_tokens = num_decoder_tokens + 1

In [82]:
#Converting the list of words into dictionary using enumerate( ) function for automatic indexing.

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
#print(input_token_index)
print(len(input_token_index.keys()))
#print(target_token_index)
print(len(target_token_index))
lines

30447
20506


,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
124725,indic2012,him name is taken in the last part of the khorana,START_ उसका नाम मुख्य द्वार के सुलेखन के अंत में खुदा _END,11,12
111177,indic2012,it has got a long market insidechata chowkwhose walls are lined with shops,START_ इसके अन्दर एक लम्बा बाजार है चट्टा चौक जिसकी दीवारें दुकानों से कतारित हैं। _END,13,16
96633,indic2012,similar language is more than doing the job,START_ साधारण भाषा मे मैथुन एक से अधिक कामक्रियाओ को संबोधित करने के लिए प्रयोग किया जाता है। _END,8,19
69492,indic2012,kokanigoanee,START_ कोंकणीगोआनी _END,1,3
41588,indic2012,akbar appointed governors to all these states,START_ अकबर ने इन राज्यों में एक एक राज्यपाल नियुक्त किया। _END,7,12
...,...,...,...,...,...
113023,indic2012,temples of nepal country,START_ नेपाली विदेश मन्त्रालय _END,4,5
39350,indic2012,chitrangada and vichitravirya were born to shantanu through satyavati,START_ शान्तनु से सत्यवती के गर्भ से चित्रांगद और विचित्रवीर्य उत्पन्न हुए थे। _END,9,14
100879,indic2012,“”sahitya lahri“” is supposed to be written by surdas,START_ साहित्य लहरी सूर की लिखी रचना मानी जाती है। _END,9,11
98796,indic2012,sursagarwhich is a popular book by surdas,START_ सूरसागर जो सूरदास की प्रसिद्ध रचना है। _END,7,9


In [84]:
#Converting the same dataset in reverse order to keep index as numeric and value as the word.

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
print(reverse_input_char_index)
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
lines = shuffle(lines)
lines.head()

{1: 'a', 2: 'aa', 3: 'aaber', 4: 'aabu', 5: 'aac', 6: 'aacharan', 7: 'aact', 8: 'aadam', 9: 'aadanthy', 10: 'aadha', 11: 'aadhi', 12: 'aadhunik', 13: 'aadi', 14: 'aadikavya', 15: 'aadim', 16: 'aadimindian', 17: 'aadimindoeuropean', 18: 'aaen', 19: 'aag', 20: 'aagara', 21: 'aagra', 22: 'aag“”', 23: 'aah', 24: 'aahamadiya', 25: 'aai', 26: 'aaimother', 27: 'aaineakbari', 28: 'aaj', 29: 'aajan', 30: 'aajmagadh', 31: 'aakarshak', 32: 'aakashdeep', 33: 'aakasher', 34: 'aakhr', 35: 'aakhus', 36: 'aakhusamarkaids', 37: 'aakraneox', 38: 'aalam', 39: 'aalekh', 40: 'aalim', 41: 'aalims', 42: 'aam', 43: 'aamar', 44: 'aamba', 45: 'aambashikandi', 46: 'aambat', 47: 'aamer', 48: 'aamray', 49: 'aanand', 50: 'aanandam', 51: 'aangghchche', 52: 'aangira', 53: 'aangkadoong', 54: 'aankhen', 55: 'aanmaya', 56: 'aanre', 57: 'aanri', 58: 'aapsall', 59: 'aapsallits', 60: 'aapte', 61: 'aara', 62: 'aaranya', 63: 'aaranyak', 64: 'aarathi', 65: 'aarati', 66: 'aardipav', 67: 'aareer', 68: 'aarhus', 69: 'aarkeneaoks

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
111933,indic2012,mullah do pija was akbars consultant,START_ मुल्लाह दो पिअज़ा अकबर के सलाहकार थे। _END,6,9
63113,indic2012,meaning of the word pura absence and past,START_ ‘पुरा शब्द का अर्थ है अनागत एवं अतीत। _END,8,10
98730,indic2012,barak hussain obamaborn augustamericas th president,START_ बराक हुसैन ओबामा जन्म अगस्त अमरीका के वें राष्ट्रपति हैं। _END,6,12
49324,indic2012,takti unicode hindi publisher,START_ तख्ती यूनिकोड हिन्दी सम्पादक _END,4,6
116821,indic2012,india based on the area is the th in world and on population india is nd,START_ भारत भौगोलिक दृष्टि से विश्व का सातवाँ सबसे बड़ा और जनसँख्या के दृष्टिकोण से दूसरा बड़ा देश है। _END,16,20
